In [1]:
#https://blogs.oracle.com/datascience/fraud-detection-using-autoencoders-in-keras-with-a-tensorflow-backend

%run -i start_spark.py --master local[*] --driver-memory 1g
#%run -i start_spark.py --master local[2] spark://192.168.0.100:7077

Creating SparkContext...
SparkContext created.
Master URL: local[*]


**Do not forget to close the session with spark.stop()**

# Model deployment

In the following the selected model is deployed in Watson Machine Learning to setup a REST API service to be offered to the client.

References:

<ol>
    <li><a href='https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-python-mnist-tutorial.html'>Watson Machine Learning Python client tutorial: Build a TensorFlow model to recognize handwritten digits using the MNIST data set</a></li>
    <li><a href='https://dataplatform.cloud.ibm.com/exchange/public/entry/view/946c86bf96596c10cc651e52501296eb'>Save, compress, and deploy a handwritten digit prediction Keras model</a></li>
</ol>

In [1]:
# Install the WML client API

#!pip uninstall  watson-machine-learning-client-V4 -y
#!pip install watson-machine-learning-client

Found existing installation: watson-machine-learning-client-V4 1.0.66
Uninstalling watson-machine-learning-client-V4-1.0.66:
  Successfully uninstalled watson-machine-learning-client-V4-1.0.66
     |████████████████████████████████| 536 kB 1.6 MB/s 


In [27]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

import coursera_common as cc
import os
import numpy as np

In [3]:

WML_CREDENTIALS = {
  "apikey": "********",
  "iam_apikey_description": "Auto-generated for key d9fb10dd-359b-4335-a749-9bb413fe9782",
  "iam_apikey_name": "Coursera_AdvAICapstone",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "********",
  "instance_id": "********",
  "url": "https://us-south.ml.cloud.ibm.com"
}

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

wml_client.version

'1.0.378'

The selected model trained over the whole dataset is exported in H5 format and compressed as zipped tarball for the deployment.

In [12]:
#compress keras model
model_filename = 'perceptron.all.v1.h5'
tar_filename = model_filename + '.tgz'
cmdstring = 'cd models && tar -zcvf ' + tar_filename + ' ' + model_filename
os.system(cmdstring)

0

In [5]:
corr_features_cols, features_cols, dl_features_cols = cc.read_feature_cols(cc.FEATURES_CONFIG_FILENAME)
print(f'label_column_names{dl_features_cols}')

Reading data/payments_enc.v6.parquet.json
label_column_names['step', 'amount', 'oldbalanceOrg', 'oldbalanceDest', 'vOrig', 'vDest', 'typeEnc']


In [17]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: 'Coursera Advanced AI Model',
    wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: 'tensorflow',
    wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: '1.15',
    wml_client.repository.ModelMetaNames.RUNTIME_NAME: 'python',
    wml_client.repository.ModelMetaNames.RUNTIME_VERSION: '3.6',
    wml_client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.2.4'}]
}

published_model_details = wml_client.repository.store_model(model=f'models/{tar_filename}', meta_props=model_props , label_column_names=dl_features_cols)     
published_model_details

{'metadata': {'guid': 'a5e20f77-5289-4427-be14-e74e8e28330b',
  'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/cabc8f15-b6a5-408f-bad1-41c87727ad81/published_models/a5e20f77-5289-4427-be14-e74e8e28330b',
  'created_at': '2020-03-24T21:06:25.894Z',
  'modified_at': '2020-03-24T21:06:25.950Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'learning_configuration_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/cabc8f15-b6a5-408f-bad1-41c87727ad81/published_models/a5e20f77-5289-4427-be14-e74e8e28330b/learning_configuration',
  'name': 'Coursera Advanced AI Model',
  'learning_iterations_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/cabc8f15-b6a5-408f-bad1-41c87727ad81/published_models/a5e20f77-5289-4427-be14-e74e8e28330b/learning_iterations',
  'feedback_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/cabc8f15-b6a5-408f-bad1-41c87727ad81/published_models/a5e20f77-5289-4427-be14-e74e8e28330b/feedback',
  'latest_version': {'url': 'https://us

In [19]:
model_uid = wml_client.repository.get_model_uid(published_model_details)
print(f'Deploy model {model_uid}')
deployment = wml_client.deployments.create(model_uid, 'Coursera Advanced AI Model (Deployed)')

deployment

Deploy model a5e20f77-5289-4427-be14-e74e8e28330b


#######################################################################################

Synchronous deployment creation for uid: 'a5e20f77-5289-4427-be14-e74e8e28330b' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='072c90fe-f912-4fca-8906-9ac680a7e5e9'
------------------------------------------------------------------------------------------------




{'metadata': {'guid': '072c90fe-f912-4fca-8906-9ac680a7e5e9',
  'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/cabc8f15-b6a5-408f-bad1-41c87727ad81/deployments/072c90fe-f912-4fca-8906-9ac680a7e5e9',
  'created_at': '2020-03-24T21:07:04.991Z',
  'modified_at': '2020-03-24T21:07:05.247Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'name': 'Coursera Advanced AI Model (Deployed)',
  'scoring_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/cabc8f15-b6a5-408f-bad1-41c87727ad81/deployments/072c90fe-f912-4fca-8906-9ac680a7e5e9/online',
  'deployable_asset': {'name': 'Coursera Advanced AI Model',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/cabc8f15-b6a5-408f-bad1-41c87727ad81/published_models/a5e20f77-5289-4427-be14-e74e8e28330b',
   'guid': 'a5e20f77-5289-4427-be14-e74e8e28330b',
   'created_at': '2020-03-24T21:07:04.941Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'mod

In [22]:
wml_client.deployments.list()

------------------------------------  -------------------------------------  ------  --------------  ------------------------  ---------------  -------------
GUID                                  NAME                                   TYPE    STATE           CREATED                   FRAMEWORK        ARTIFACT TYPE
072c90fe-f912-4fca-8906-9ac680a7e5e9  Coursera Advanced AI Model (Deployed)  online  DEPLOY_SUCCESS  2020-03-24T21:07:04.991Z  tensorflow-1.15  model
------------------------------------  -------------------------------------  ------  --------------  ------------------------  ---------------  -------------


In [23]:
scoring_endpoint = wml_client.deployments.get_scoring_url(deployment)
print(f'Scoring endpoint: {scoring_endpoint}')

Scoring endpoint: https://us-south.ml.cloud.ibm.com/v3/wml_instances/cabc8f15-b6a5-408f-bad1-41c87727ad81/deployments/072c90fe-f912-4fca-8906-9ac680a7e5e9/online


The deployed model is therefore tested

In [34]:
if os.path.exists( cc.AUTOENC_X_TEST ):
    print('Loading ' + cc.AUTOENC_X_TEST)
    x_test = np.load( cc.AUTOENC_X_TEST )
    print('Loading ' + cc.AUTOENC_Y_TEST)
    y_test = np.load( cc.AUTOENC_Y_TEST )

Loading data/autoencoder.x_test.v2.npy
Loading data/autoencoder.y_test.v2.npy


In [39]:
print('Testing deployed model...')
payment_data = x_test[0].tolist()

scoring_payload = {'values': [payment_data]}
scores = wml_client.deployments.score(scoring_endpoint, scoring_payload)
print('Score: Predicted {} - Actual {}'.format(scores['values'][0], y_test[0]))

Testing deployed model...
Score: Predicted [0.0] - Actual 0


In [46]:
it = np.where(y_test == 1)

y_test_t = y_test[it]

print(f'Fraudolent transaction: [{x_test_t[0]}]')

scoring_payload = {'values': [x_test_t[0].tolist()]}
scores = wml_client.deployments.score(scoring_endpoint, scoring_payload)

fraud_scoring_threshold = 0.1

is_fraud = scores['values'][0][0] > fraud_scoring_threshold

print(f'Prediction: Is Fraud? {is_fraud}')

Fraudolent transaction: [[-0.29694012 -0.28888892 -0.32524011  0.1595821  -0.71474652  1.98121586]]
Prediction: Is Fraud? True
